In [1]:
! ./.g


[master 3fa9a7d] web
 1 file changed, 191 insertions(+), 211 deletions(-)
Counting objects: 3, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 14.94 KiB | 2.99 MiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   f06fcf3..3fa9a7d  master -> master


In [1]:
ls


 8djv8.pkl                            kqqtv.pkl
 94f6b.pkl                            kv6n2.pkl
 afuahirsch.csv                       kvf22.pkl
 davidsmith.csv                       liberation_femicide.ipynb
 edt72.pkl                            links_2020_8.txt
 eghyd.pkl                            links.txt
 egqhf.pkl                            matplotlib.ipynb
 ej7f5.pkl                            nickcohen.csv
 ejdxt.pkl                            None0000000.png
 et8zg.pkl                            owen-jones.csv
 etenk.pkl                            quad_forms.ipynb
 eyhx8.pkl                            README.md
 eytkx.pkl                            rebeccasolnit.csv
 f3ke9.pkl                            simontisdall.csv
 f4a3q.pkl                            tags
 f4vfe.pkl                            tbca8.pkl
 failed.txt                           tdvxp.pkl
 femicides2.csv                       te5tn.pkl
 femicides3.csv                       tedzk.pkl
 femicides.csv            

In [ ]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [60]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','a') as fp:
    fp.write('\n'.join(lks))

In [2]:
import re 
import requests
import pickle
import json

In [9]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/21/joe-biden-democrat-convention-nice-guy-trump-fight'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/23/the-cruel-exams-algorithm-has-laid-bare-the-unfairness-at-the-heart-of-our-schools'
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/01/us-mainstream-right-vigilante-terror'
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/31/politicians-covid-19-statistics-statisticians'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/may/10/retail-politics-movements-polarising-projects-promise'

In [11]:
article_url = 'https://www.theguardian.com/commentisfree/2020/sep/29/rival-scientists-lockdowns-scientific-covid-19'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/08/johnson-and-sturgeon-will-need-a-better-philosophy-than-precaution-to-rid-us-of-covid'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/10/continual-local-lockdowns-answer-covid-control'
article_url = 'https://www.theguardian.com/commentisfree/2020/oct/11/the-rebel-scientists-cause-would-be-more-persuasive-if-it-werent-so-half-baked'

article_url = 'https://www.theguardian.com/commentisfree/2021/apr/03/undermining-the-astrazeneca-jab-is-a-dangerous-act-of-political-folly#comment-148440261'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/09/boris-johnson-brexit-belfast-violence-eu-good-friday-agreement'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/02/culture-war-government-race-report'
article_url = 'https://www.theguardian.com/commentisfree/2021/mar/18/britain-sunny-vaccine-optimism-april-delivery-delay'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/14/boris-johnson-is-telling-scotland-that-the-union-is-no-longer-based-on-consent'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/15/britain-harness-social-sciences-covid-pandemic-deprivation'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/16/government-race-report-evidence#comment-148662066'
article_url ='https://www.theguardian.com/commentisfree/2021/apr/20/britain-falling-birthrate-covid-pandemic-conservatives-removed-support-for-parents'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/21/bmi-eating-disorder-pandemic-treatment'
article_url = 'https://www.theguardian.com/commentisfree/2021/apr/30/government-delays-covid-public-inquiry-health-crisis'
article_url = 'https://www.theguardian.com/commentisfree/2021/may/07/humiliation-hartlepool-smalltown-detective-di-starmer'
article_url ='https://www.theguardian.com/commentisfree/2021/aug/06/uk-government-experts-vaccinating-under-16s'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/01/there-is-a-case-for-vaccine-passports-but-ministers-are-failing-to-make-it'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/09/america-is-flying-blind-when-it-comes-to-the-delta-variant'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/10/covid-vaccines-britain-scientific-solution-jabs'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/13/denial-anti-vaxxers-climate-sceptics'
article_url = 'https://www.theguardian.com/commentisfree/2018/apr/11/bbc-andrew-neil-media-politics'

article_url = 'https://www.theguardian.com/commentisfree/2021/aug/15/vaccine-passports-are-less-a-threat-to-liberty-than-a-mark-of-solidarity'
article_url = 'https://www.theguardian.com/theobserver/commentisfree/2021/aug/14/will-we-reach-herd-immunity-for-the-new-coronavirus'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/15/vaccine-hesitancy-broken-relationship-state-conspiracy-theorists'
article_url = 'https://www.theguardian.com/commentisfree/2021/aug/17/dominic-raab-news-foreign-secretary-taliban-kabul'
article_url= 'https://www.theguardian.com/commentisfree/2021/dec/11/so-which-of-these-politicians-is-a-neoliberal-not-one-of-them'

article_url = 'https://www.theguardian.com/commentisfree/2021/dec/12/omicron-is-sneaky-it-could-be-fatal-for-us-or-for-our-faith-in-government'

In [13]:
article_url ='https://www.theguardian.com/commentisfree/2021/dec/06/google-silicon-valley-ai-timnit-gebru'
article_url = 'https://www.theguardian.com/commentisfree/2021/dec/20/rightwingers-us-social-change-coming'
article_url = 'https://www.theguardian.com/commentisfree/2021/dec/22/omicron-covid-restrictions-young-people'
article_url = 'https://www.theguardian.com/politics/2022/jan/01/sir-tim-brighouse-many-hold-gove-responsible-expert-educator-sets-out-whats-gone-wrong-with-britains-schools'
article_url ='https://www.theguardian.com/commentisfree/2022/jan/10/colston-four-britain-apology-for-past'
article_url = 'https://www.theguardian.com/commentisfree/2022/jan/21/democrats-shape-up-bidens-presidency-trumpian-sequel'
article_url = 'https://www.theguardian.com/commentisfree/2022/jan/28/joe-rogan-neil-young-spotify-streaming-service'

In [7]:
article_url = 'https://www.theguardian.com/commentisfree/2022/mar/04/ukraine-eu-membership-zelenskiy-european-values-war'
article_url = 'https://www.theguardian.com/commentisfree/2022/feb/27/liberal-democracies-must-defend-their-values-and-show-putin-that-the-west-isnt-weak'
article_url = 'https://www.theguardian.com/commentisfree/2022/mar/06/progress-women-international-womens-day-labour-party'
article_url = 'https://www.theguardian.com/commentisfree/2022/mar/09/vladimir-putin-west-downfall-dictator-democracies'
article_url = 'https://www.theguardian.com/commentisfree/2022/mar/26/britain-clocks-go-forward-eu-us-hour-changes'
article_url = 'https://www.theguardian.com/commentisfree/2022/mar/27/for-brazen-cynicism-i-have-seen-nothing-like-sunaks-plan-in-40-years'

In [3]:
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/01/straight-women-tv-shows-lesbians-sally-wainwright'
article_url ='https://www.theguardian.com/commentisfree/2022/apr/06/united-nations-putin-ukraine-catastrophic'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/10/stench-entitlement-now-oozing-from-rishi-sunaks-home-as-well-as-boris-johnsons'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/10/orban-le-pen-voters-sening-chill-message-to-europes-beleagured-centre'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/11/france-populism-gaining-ground-revolution'

article_url = 'https://www.theguardian.com/commentisfree/2022/apr/12/herd-immunity-covid-reinfection-virus-world'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/17/whoever-ends-up-in-elysee-palace-europeanism-is-already-on-the-way-out'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/17/nato-should-talk-less-and-do-more-or-ukraine-will-be-torn-apart-bit-by-bit'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/20/france-emmanuel-macron-president-far-right-marine-le-pen'
article_url ='https://www.theguardian.com/commentisfree/2022/apr/23/france-vote-le-pen-macron-tv-debate-far-right'
article_url = 'https://www.theguardian.com/commentisfree/2022/apr/22/hips-dont-lie-liam-gallagher-replacement-surgery-older-bodies'

article_url = 'https://www.theguardian.com/commentisfree/2022/apr/25/french-left-emmanuel-macron-marine-le-pen-far-right-president'
article_url = 'https://www.theguardian.com/commentisfree/2022/may/09/ncuti-gatwa-doctor-who-jodie-whittaker-whovians'
article_url = 'https://www.theguardian.com/world/commentisfree/2022/apr/28/france-election-divide-campaign-split'

article_url ='https://www.theguardian.com/commentisfree/2022/may/13/phone-north-sea-rebekah-vardy-coleen-rooney-libel-case'

article_url = 'https://www.theguardian.com/commentisfree/2022/jun/09/west-victory-in-ukraine-escalation-war'
article_url = 'https://www.theguardian.com/commentisfree/2022/jun/24/overturning-roe-story-is-women-unfreedom'
article_url = 'https://www.theguardian.com/commentisfree/2022/jul/01/nicola-sturgeon-independence-referendum-britain'
article_url = 'https://www.theguardian.com/commentisfree/2022/jul/11/would-be-mothers-parent-solo-dating-apps-children'
article_url = 'https://www.theguardian.com/commentisfree/2022/jul/27/womens-football-is-not-just-a-spin-off-of-the-mens-game-its-great-in-its-own-right'

In [ ]:


key = my_keys.keys['guardian']

In [4]:
#scrape the page and get the short url
# 15/8/21  this failed and gave /p/temp
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)
uu = r.text
print(short_url)

#the short url allows us to access the discussion
# 15/8/21 api.nextgen.guardianapps.co.uk/discussion/ failed on 
# https://discussion.theguardian.com/discussion-api/discussion/p/tfmxg
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn,'wb') as fp:
    pickle.dump(json_data,fp)

/p/mv97t
getting comments page no. 1
there are 7 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
getting comments page no. 7
archiving to mv97t.pkl


In [2]:
import pickle

json_data = pickle.load(open('k4vhe.pkl','rb'))

---

## remember each page of comments has been stored separately

In [9]:
dd = json.loads(json_data[0])
len(json_data)

14

In [19]:
xx = json.loads(json_data[0])
xx['commentsHtml'][:8000]

'\n<div class="d-discussion d-discussion--recommendations-open u-cf"\n     data-read-only="false" >\n\n    \n    \n        <div class="d-discussion__size-message">\n            Due to the large number of comments, they are being shown 100 per page.\n        </div>\n    \n\n\n    <ul class="d-thread d-thread--comments js-new-comments">\n\n        \n            \n\n<li class="d-comment\n            d-comment--top-level d-comment-staff"\n           id="comment-154007965"\n           data-comment-id="154007965"\n           data-comment-author="CommunityMod"\n           data-comment-replies="0"\n           data-comment-timestamp="2022-01-01T18:00:47.000Z"\n           data-comment-author-id="2902769"\n           data-comment-highlighted="false"\n           itemscope itemtype="http://schema.org/Comment">\n\n    <div class="d-comment__inner d-comment__inner--top-level">\n        <div class="d-comment__meta">\n            <span class="d-comment__avatar-wrapper">\n                <img src="https

In [ ]:
https://discussion.theguardian.com/comment-permalink/154003870

In [136]:
print( xx['commentsHtml'][200:700])

 class="d-comment
            d-comment--top-level"
           id="comment-144422422"
           data-comment-id="144422422"
           data-comment-author="Hushing"
           data-comment-replies="0"
           data-comment-timestamp="2020-10-11T14:25:06.000+01:00"
           data-comment-author-id="12407494"
           data-comment-highlighted="false"
           itemscope itemtype="http://schema.org/Comment">

    <div class="d-comment__inner d-comment__inner--top-level">
        <div class="


In [8]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')
pcid = re.compile('\sid="comment-(.*?)"')
pp = re.compile('<li class="d-comment(.*?)>',re.DOTALL)

auths = []
md = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh), 
                           pid.findall(hh)
                          )))
    md.extend(pp.findall(hh))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [10]:
auths[:10]

[('123654a', '14312750'),
 ('1key2kan', '104749605'),
 ('1Love1Heart', '100796168'),
 ('45022', '3426044'),
 ('48percenter', '2847558'),
 ('Abedonders', '3883075'),
 ('abstrusus', '15533337'),
 ('adamvarhy', '106853452'),
 ('ade1973', '14874802'),
 ('AlexanderGH', '4503360')]

In [102]:
#print( '\n'.join([x for x in md if 'pinkie' in x.lower()]))

https://discussion.theguardian.com/comment-permalink/144375739

In [8]:
auths = dict(auths)

In [13]:
comments_url

'https://api.nextgen.guardianapps.co.uk/discussion//p/yydf9.json'

In [5]:
from bs4 import BeautifulSoup
import lxml

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    
    auth = [ x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    #this was a pain to do really
    #the problem is blocked comments
    comment_id = [ x['id'][len('comment-'):] for x in soup.findAll('li')]
 
    
    ss = '##{}\nhttps://discussion.theguardian.com/comment-permalink/{}\n{}\n\n{}'
    return [ ss.format(a,b,d,c.strip())  
                   for a,b,d,c in zip(auth, comment_id, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]


from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

In [14]:
all_comments[0]

'##COMMUNITYMOD\nhttps://discussion.theguardian.com/comment-permalink/157514033\nhttps://profile.theguardian.com/user/id/2902769\n\nHi,\nComments will close at 15:45.'

# Random data hacking

In [6]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##JOHNNIEAYSGARTH', 14),
 ('##BANKER62', 8),
 ('##17LUFTBALLONS', 7),
 ('##DFIC1999', 7),
 ('##LALTHEGAL2021', 6),
 ('##FARABUNDOVIVE', 6),
 ('##HACKERJACK79', 6),
 ('##BARONVONDOMINIC', 5),
 ('##JFORBES', 5)]

In [24]:
! ./.g


[master 26cbffe] web
 5 files changed, 228 insertions(+), 271 deletions(-)
 create mode 100644 yb8ez.pkl
 create mode 100644 yddvx.pkl
 create mode 100644 ye5m7.pkl
 create mode 100644 yjvf7.pkl
Counting objects: 7, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 738.28 KiB | 2.44 MiB/s, done.
Total 7 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   f914f6e..26cbffe  master -> master


In [30]:
print(all_comments[0][:200])

##STILT
https://discussion.theguardian.com/comment-permalink/155280552
https://profile.theguardian.com/user/id/2397205

This comment was removed by a moderator because it didn't abide by our
         


'france-populism-gaining-ground-revolution'

In [7]:
import os

all_comments.sort(key=lambda x : -len(x) )
txt = '\n\n'.join(all_comments[:20])
print(txt)
with open(os.environ['HOME']+'/tmp.txt','w') as fp:
    fp.write('\n\n'.join(all_comments[:20]))
with open(os.environ['HOME']+'/{}.txt'.format(article_url.split('/')[-1]),'w') as fp:
    fp.write('\n\n'.join(all_comments[:20]))

##JOHNBARNESONTOAST
https://discussion.theguardian.com/comment-permalink/157847286
https://profile.theguardian.com/user/id/2999750

I think a premise touched on in this article is valid - that the women's game needs to be approached and appreciated in its own terms, rather than the continued comparatives with the men's game.
They are entirely different 'products' (for lack of a better word)Leaving aside the braying from some quarters who will always look to denigrate the sport in relation to the men's iteration, we have in recent times seen an increasing tendency to draw comparisons in the areas that the women's game purportedly 'bests' that of the men's - in sporting integrity, honesty, its family-friendly nature.I contend that this route is a path to failure. Ultimately, those 'positive' characteristics broadly equate to a simple mirroring of the women's game's progress along the timeline of professionalism; there is less at stake for the players and the fans; there is less pressure.

In [6]:
! ./.g

[master f06fcf3] web
 4 files changed, 341 insertions(+), 216 deletions(-)
 create mode 100644 ymmym.pkl
 create mode 100644 yy437.pkl
 create mode 100644 yydf9.pkl
Counting objects: 6, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 598.61 KiB | 1.82 MiB/s, done.
Total 6 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 2 local objects.
To https://github.com/macbuse/GUARDIAN.git
   26cbffe..f06fcf3  master -> master


In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
#nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [10]:
target = 'decolon'
target = 'science'
target = 'irish'
target = 'mandela'
target = 'abolit'
target = 'whittaker'
target = 'capaldi'


xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )

#print( '\n\n'.join(xx).replace(target, ) )
pxx = re.compile(target, re.I)
print( pxx.sub('**' + target.upper() + '**', 
              '\n\n'.join(xx)) )
               
#print( '\n\n'.join(xx) )


##LIBBETS
https://discussion.theguardian.com/comment-permalink/156335914
https://profile.theguardian.com/user/id/12302017

Peter **CAPALDI** sums it up well for me.'People always ask me what it is about Doctor Who that appeals so broadly. The answer I would like to give, and which I’m discouraged from giving because it is not useful in the promotion of a brand, is that it’s about death. It has a very powerful death motif in it which is that the central character dies. I think that is one of its most potent mysteries because somewhere in that people see what happens in life. You have loved ones and then they go, but you must carry on.'
Here's to carrying on. Best wishes to Ncuti Gatwa, I can't wait.

##MACHINEHEAD
https://discussion.theguardian.com/comment-permalink/156338017
https://profile.theguardian.com/user/id/1642806

I’m not convinced Whitaker was a great Doctor to be honest…middling I’d say but like **CAPALDI** before her she was not gifted with the greatest storylines much of t

In [20]:
(26/31) / (74/69)

0.7820401046207497

In [64]:
23**2

529

## 

In [21]:
84 + 127

211

In [24]:
84/211/.31

1.2842073077511085

In [25]:
127/211/.69

0.8723126588364586

In [26]:
1.28/.87

1.471264367816092